Check completeness of ConfIDent metadata:
Collect metadata per DOI from API-response, calculate percenate of data missing per field, export to excel 

Metadata fields not included, since not applicable / never filled yet according to API query: 
sizes, formats, version (count 0), rightsList (count 1)

Issues:
Very repetetive (with variants)! Kapseln möglich?



To be done, if helpful:

- Group entries from same dict in json in dicts here as well
- Calculate percentages of all missing entries
- Other form of display/export of results more useful?
- Include type of description in collected data

In [21]:
import requests
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
plt.close("all")

# Define the API URL
URL = "https://api.datacite.org/dois"

# Define the query parameters
params = {
    "prefix": "10.25798",
    "page[number]": 1,
    "page[size]": 1
}

# Initialize lists to store data, with repeatable entries as sublist per DOI

# Of time stamps, use "registered" to reflect when the DOI was created with currently missing metadata 
# Assumption: updates add metadata rather than removing them
registered = []
dois = []
# URL of landing page
urls = []
# Entry "creator" in DC-MD-Schema means "hosting or initialising institution of event (series)" in ConfIDent
creators = []
title = []
publisher = []
# Entry "publication year" in DC-MD-Schema means "year the event took place" in ConfIDent 
publication_year = []
restype = []
restypeg = []
subjects = []
contributors = []
contributor_ids = []
dates = []
alternative_identifiers = []
related_identifiers = []
description = []
# Stores geolocation property (type of geolocation given)
geo_locations = []
language = []





# Function to check if string already is in list of strings, independent of capitalization. Return None (no dublicate) if list is empty
def check_for_dublicate (list, string):
    try:
        for j in range(len(list)):
            if list[j].lower() != string.lower():
                pass
            else: 
                return True
    except ValueError:
        return None

# Fetch n pages of API-response and collect entries in "data" from single DOIs
def fetch_data():    
    response = requests.get(URL, params=params)
    antwort = response.json()
    
    for item in antwort["data"]:
        attributes = item["attributes"]
        
    # Collect non-repeatable entries
        registered.append(attributes["registered"])
        dois.append(attributes["doi"])
    # If not given (though mandatory), the following values could be None-type
    # If error occurs, cf . "types"
        urls.append(attributes["url"])
        publication_year.append(attributes["publicationYear"])
        publisher.append(attributes["publisher"])

        if type(attributes["types"]["resourceType"]) == str:
            restypeg.append(attributes["types"]["resourceType"])
        elif attributes["types"]["resourceType"] == None:
            restypeg.append([])
        
        if type(attributes["types"]["resourceTypeGeneral"]) == str:
            restypeg.append(attributes["types"]["resourceTypeGeneral"])
        elif attributes["types"]["resourceTypeGeneral"] == None:
            restypeg.append([])
                            
        # Collect whole entry "dates" as list of possibly repeated entry "date" (with different date types) assigend to single DOI. 
        # Shoud reflect duration of event. Should return empty list if no entry was made.
        dates.append(attributes["dates"])
        
        # As of 20240426, language entry is either str (from controlled list) or None, 
        # but is planned to allow multiple entries for language
        if type(attributes["language"]) == str:
            language.append(attributes["language"])
        elif attributes["language"] == None:
            language.append([])
        else:
            language.append("json has changed, someone needs to update the code")
         
        # Collect multible entries from fields with (repeatable) subfields

        eintrag_urheber_item = (attributes["creators"])
        urheber_item = []
         # In case no creator is given: attributes["creators"] == [], the following then is "for i in range(0, 0)" and not executed 
         # => no index error, list remains as empty list
        for i in range(len(eintrag_urheber_item)):
            urheber = eintrag_urheber_item[i]["name"]
            if check_for_dublicate(urheber_item, urheber) == None:
                urheber_item.append(urheber)
        creators.append(urheber_item)

        eintrag_titel_item = (attributes["titles"])
        titel_item = []
        for i in range(len(eintrag_titel_item)):
            titel = eintrag_titel_item[i]["title"]
            if check_for_dublicate(titel_item, titel) == None:
                titel_item.append(titel)
        title.append(titel_item)
        
        eintrag_subjects_item = (attributes["subjects"])
        schlagwoerter_item = []
        for i in range(len(eintrag_subjects_item)):
            schlagwort = eintrag_subjects_item[i]["subject"]
            if check_for_dublicate(schlagwoerter_item, schlagwort) == None:
                # Capitalize freetext entries to avoid splitting of subject count due to inconsistent capitalization 
                if schlagwort[:4] != "FOS:":
                    schlagwort = schlagwort.capitalize()
                schlagwoerter_item.append(schlagwort)
        subjects.append(schlagwoerter_item)
        
        eintrag_contributors_item = (attributes["contributors"])
        contributors_item = []
        contributorids_item = []
        for i in range(len(eintrag_contributors_item)):
            contributor = eintrag_contributors_item[i]["name"]
            contributors_item.append(contributor)
            for j in range(len(eintrag_contributors_item[i]["nameIdentifiers"])):
                contributorid = eintrag_contributors_item[i]["nameIdentifiers"][j].get("nameIdentifier")
                if contributorid:
                    contributorids_item.append(contributorid)
                else:
                    contributorids_item.append([])
        contributors.append(contributors_item)
        contributor_ids.append(contributorids_item)

        eintrag_relidents_item = (attributes["relatedIdentifiers"])
        relidents_item = []
        for i in range(len(eintrag_relidents_item)):
            relident = eintrag_relidents_item[i]["relatedIdentifier"]
            relidents_item.append(relident)
        related_identifiers.append(relidents_item)

        eintrag_descriptions_item = (attributes["descriptions"])
        descriptions_item = []
        for i in range(len(eintrag_descriptions_item)):
            beschreibung = eintrag_descriptions_item[i]["description"]
            # Fetch only first 100 characters of description
            descriptions_item.append(beschreibung[:100])
        description.append(descriptions_item)

        eintrag_geolocations_item = (attributes["geoLocations"])
        geolocations_item = []
        for i in range(len(eintrag_geolocations_item)):
            ortsangabe = list(eintrag_geolocations_item[i].keys())
            geolocations_item.append(ortsangabe)
        geo_locations.append(geolocations_item)

        eintrag_altidents_item = (attributes["identifiers"])
        altidents_item = []
        for i in range(len(eintrag_altidents_item)):
            altid = eintrag_altidents_item[i]["identifier"]
            altidents_item.append(altid)
        alternative_identifiers.append(altidents_item)
        
    return antwort["links"].get("next")


while params["page[number]"] < 2: 
#while True:
    next_page = fetch_data()
    # Check if there is a next page to fetch, if yes, count up page number of query
    if next_page == None:
        break
    else: 
        params["page[number]"] += 1


In [18]:
# Calculate percentage of metadata sets with specific entry missing
# for all entries listed except "DOI" and "registered" (always 100%)

number_dois = len(dois)
md_fields = [registered,
             dois,
             urls,
             creators,
             title,
             publisher,
             publication_year,
             restype,
             restypeg,
             subjects,
             contributors,
             contributor_ids,
             dates,
             alternative_identifiers,
             related_identifiers,
             description,
             geo_locations,
             language]

for field in md_fields: 
    no_entry = 0
    for i in range(len(field)):
        if field[i] == []:
            no_entry +=1
    perc_no_entry = round(((no_entry / number_dois) * 100), 2)
    field.insert(0, f"Percentage missing: {perc_no_entry}")
 

In [19]:
# Create a DataFrame from the extracted data per DOI

# Untrancate list - does not work with broad columns
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 4000)

df = pd.DataFrame({
    "Registration Date" : registered,
    "DOI": dois,
    "URL": urls,
    "Institution": creators,
    "Title": title,
    "Publisher": publisher,
    "Year of Event": publication_year,
    "Other Dates of Event": dates,
    "Resource Type General": restypeg,
    "Resource Type": restype,
    "Subject": subjects,
    "Contributors": contributors,
    "Contributor IDs": contributor_ids,
    "Related Identifiers": related_identifiers,
    "Description": description,
    "Geo Location": geo_locations, 
    "Language": language,
    "Alternative ID": alternative_identifiers
})


In [20]:
#print(df)
# Export DataFrame to Excel file
df.to_excel("ConfIDent_MD-completeness.xlsx")
print("fertig!")

fertig!
